In [ ]:
### para manipulação dos dados em python
import pandas as pd
import numpy as np
import os
import math
import seaborn as sns
import matplotlib as plt
import ipykernel

### para obter estatisticas de teste
import scipy.stats as stats
from scipy.stats import norm
from sklearn.preprocessing import LabelEncoder

from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.model_selection import train_test_split, ShuffleSplit, KFold, cross_validate
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.dummy import DummyRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error

In [ ]:
#Caminho e leitura do Dataframe
df = pd.read_csv('../data/raw/Orange_Quality_Data.csv')

#Análise das 10 primeiras linhas 
df.head(10)

In [ ]:
# Leitura do CSV
df_dicionario_dados = pd.read_csv('../data/external/dicionario_de_dados_laranjas.csv')

#Análise das 10 primeiras linhas 
df_dicionario_dados.head(10)

In [ ]:
#Checagem de nomes de coluna para facilitar elaboração do código 
#print(df_dicionario_dados.columns)

In [ ]:
#Organização da busca pelo tipo de variável

variavel_alvo = 'Quality (1-5)'

# Renomeando colunas (opcional)
df_dicionario_dados = df_dicionario_dados.rename(columns={'Variável': 'Variavel', 'Subtipo': 'Subtipo'})


variavel_nominal = (
    df_dicionario_dados
    .query("Subtipo == 'Nominal' and Variavel != @variavel_alvo")
    .Variavel
    .to_list()
)
print(f"Variáveis qualitativo: nominal")
print(variavel_nominal)
print(" ")

variavel_quanti_continua = (
    df_dicionario_dados
    .query("Subtipo == 'Contínua' and Variavel != @variavel_alvo")
    .Variavel #Nome da coluna que tem as variáveis
    .to_list() #Método que lista
)
print(f"Variável quantitativo: Contínua")
print(variavel_quanti_continua)
print(" ")
variavel_quanti_discreta = (
    df_dicionario_dados
    .query("Subtipo == 'Discreto' and Variavel != @variavel_alvo")
    .Variavel #Nome da coluna que tem as variáveis
    .to_list() #Método que lista
)
print(f"Variável quantitativo: Discreto")
print(variavel_quanti_discreta)
print(" ")

x = df.drop(columns=[variavel_alvo])
y = df[variavel_alvo]

## Pré-processamento

In [ ]:
processamento_nominal = Pipeline(steps=[
    ("missing", SimpleImputer(strategy='most_frequent')), # tratamento para dados faltantes
    ("encoding", OneHotEncoder(sparse_output=False)), # codificação de variáveis
])

processamento_continuos = Pipeline(steps=[
    ("missing", SimpleImputer(strategy='mean')), #Tratamento de dados faltantes com substituição pela MÉDIA
    ("normalization", StandardScaler()), #Normalização de variáveis
     
])
processamento_discreto = Pipeline(steps=[
    ("missing", KNNImputer()), #Tratamento de dados faltantes com substituição pela MEDIANA
    ("normalization", StandardScaler()), #Normalização de variáveis
     
])

# Combinar transformações e aplicar 

In [ ]:
pre_processador = ColumnTransformer([
    ("Contínua", processamento_continuos, variavel_quanti_continua),
    ("Discreto", processamento_discreto, variavel_quanti_discreta),
    ("Nominal", processamento_nominal, variavel_nominal)
])

df_preprocessed = pre_processador.fit_transform(df)

df_preprocessed

## Método de validação cruzada

In [ ]:
#Dividindo a base de dados para o treinamento e teste

monte_carlo = ShuffleSplit(n_splits=10, test_size=0.2, random_state=42)

# Modelos a serem avaliados
modelos = [DummyRegressor(strategy='mean'), LinearRegression(), KNeighborsRegressor(n_neighbors=5), SVR()]

# Métricas a serem utilizadas
metricas = [
    'neg_mean_absolute_error',
    'neg_mean_squared_error',
    'neg_mean_absolute_percentage_error',
    'r2',
]

# Loop para avaliação dos modelos
resultados_totais = None
for modelo in modelos:
    nome_modelo = modelo.__class__.__name__
    print(f"Rodando para o modelo: {nome_modelo}")
    pipeline = Pipeline(steps=[
        ('preprocessor', pre_processador),
        ('model', modelo),
    ])
    pontuacoes = cross_validate(
        pipeline, x, y,
        scoring=metricas,
        cv=monte_carlo
    )
    resultados_modelo = pd.DataFrame(pontuacoes)
    resultados_modelo['model'] = nome_modelo
    if resultados_totais is None:
        resultados_totais = resultados_modelo
    else:
        resultados_totais = pd.concat([resultados_totais, resultados_modelo])

# Resumo dos resultados
#print(resultados_totais.groupby('model').agg(['mean', 'std']).T)

In [ ]:
resultados_totais.groupby('model').agg(['mean', 'std']).T